In [1]:
from google.colab import drive
drive.mount('./gdrive')

Mounted at ./gdrive


In [2]:
import numpy as np 
import pandas as pd 
import random
import cv2
import os

In [3]:
os.chdir("/content/gdrive/MyDrive/ML Project/Mushroom Research Paper")

In [4]:
clss = os.listdir()
clss

['Agaricus Bisporus',
 'Death Cap',
 'Galerina Autumnalis',
 'Oyster',
 'data_files',
 'make_generator_dataset.ipynb',
 'test2.ipynb',
 'training_without_data_generator_transfered_learning.ipynb',
 'training_with_data_generator_custom_model_1.ipynb',
 'training_with_data_generator_custom_model_2.ipynb']

In [5]:
from PIL import Image
import matplotlib.pyplot as plt


import cv2
from sklearn.utils import shuffle

from keras.utils import np_utils

In [6]:
def load_samples(csv_file):
    data = pd.read_csv(os.path.join('data_files',csv_file))
    data = data[['FileName', 'Label', 'ClassName']]
    file_names = list(data.iloc[:,0])
    # Get the labels present in the second column
    labels = list(data.iloc[:,1])
    samples=[]
    for samp,lab in zip(file_names,labels):
        samples.append([samp,lab])
    return samples

In [7]:
data_path = 'mushroom_recognition_train.csv'
samples = load_samples(data_path)

In [8]:
print (samples[0:5])

[['/content/gdrive/My Drive/ML Project/Mushroom Research Paper/Agaricus Bisporus/pic_004.jpg', 0], ['/content/gdrive/My Drive/ML Project/Mushroom Research Paper/Agaricus Bisporus/pic_007.jpg', 0], ['/content/gdrive/My Drive/ML Project/Mushroom Research Paper/Agaricus Bisporus/pic_003.jpg', 0], ['/content/gdrive/My Drive/ML Project/Mushroom Research Paper/Agaricus Bisporus/pic_005.jpg', 0], ['/content/gdrive/My Drive/ML Project/Mushroom Research Paper/Agaricus Bisporus/pic_014.jpg', 0]]


In [9]:
def shuffle_data(data):
    data = shuffle(data)#,random_state=2)
    return data

In [10]:
def preprocessing(img,label):
    img = cv2.resize(img,(300,300))
    img = img/255
    label = np_utils.to_categorical(label, 4)
    return img,label

In [11]:
def data_generator(samples, batch_size=32,shuffle_data=True,resize=224):
    """
    Yields the next training batch.
    Suppose `samples` is an array [[image1_filename,label1], [image2_filename,label2],...].
    """
    num_samples = len(samples)
    while True: # Loop forever so the generator never terminates
        samples = shuffle(samples)

        # Get index to start each batch: [0, batch_size, 2*batch_size, ..., max multiple of batch_size <= num_samples]
        for offset in range(0, num_samples, batch_size):
            # Get the samples you'll use in this batch
            batch_samples = samples[offset:offset+batch_size]

            # Initialise X_train and y_train arrays for this batch
            X_train = []
            y_train = []

            # For each example
            for batch_sample in batch_samples:
                # Load image (X) and label (y)
                img_name = batch_sample[0]
                label = batch_sample[1]
                img =  cv2.imread(os.path.join("",img_name))
                
                # apply any kind of preprocessing
                img,label = preprocessing(img,label)
                # Add example to arrays
                X_train.append(img)
                y_train.append(label)

            # Make sure they're numpy arrays (as opposed to lists)
            X_train = np.array(X_train)
            y_train = np.array(y_train)

            # The generator-y part: yield the next training batch            
            yield X_train, y_train

In [12]:
# this will create a generator object
train_datagen = data_generator(samples,batch_size=8)

x,y = next(train_datagen)
print ('x_shape: ', x.shape)
print ('labels shape: ', y.shape)
print ('labels: ', y)

x_shape:  (8, 300, 300, 3)
labels shape:  (8, 4)
labels:  [[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]]


In [13]:
train_data_path = 'mushroom_recognition_train.csv'
test_data_path = 'mushroom_recognition_test.csv'

train_samples = load_samples(train_data_path)
test_samples = load_samples(test_data_path)

num_train_samples = len(train_samples)
num_test_samples = len(test_samples)

print ('number of train samples: ', num_train_samples)
print ('number of test samples: ', num_test_samples)

number of train samples:  1412
number of test samples:  799


In [14]:
# Create generator
batch_size = 10
train_generator = data_generator(train_samples, batch_size=32)
validation_generator = data_generator(test_samples, batch_size=32)

In [3]:
# import the necessary modules from the library
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Activation, MaxPooling2D, Dropout
from keras.layers import BatchNormalization

input_shape = (300,300,3)
print (input_shape)

model = Sequential()
model.add(Conv2D(filters=64, kernel_size=(5, 5), input_shape=input_shape, activation='relu'))
model.add(BatchNormalization(axis=3))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization(axis=3))
model.add(Dropout(0.1))
model.add(Conv2D(filters=128, kernel_size=(5, 5), activation='relu'))
model.add(BatchNormalization(axis=3))
model.add(Conv2D(filters=128, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization(axis=3))
model.add(Dropout(0.1))
model.add(Conv2D(filters=256, kernel_size=(5, 5), activation='relu'))
model.add(BatchNormalization(axis=3))
model.add(Conv2D(filters=256, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization(axis=3))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

(300, 300, 3)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 296, 296, 64)      4864      
_________________________________________________________________
batch_normalization_8 (Batch (None, 296, 296, 64)      256       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 292, 292, 64)      102464    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 146, 146, 64)      0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 146, 146, 64)      256       
_________________________________________________________________
dropout_5 (Dropout)          (None, 146, 146, 64)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 142,

In [37]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 296, 296, 64)      4864      
_________________________________________________________________
activation (Activation)      (None, 296, 296, 64)      0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 292, 292, 64)      102464    
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 146, 146, 64)      0         
_________________________________________________________________
activation_1 (Activation)    (None, 146, 146, 64)      0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 146, 146, 64)      0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 1364224)          

In [38]:
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='sgd')

In [ ]:
history_mdl1=model.fit_generator(
        train_generator,
        steps_per_epoch=num_train_samples // batch_size,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=num_test_samples // batch_size)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
141/141 [==============================] - 7171s 51s/step - loss: 1.3674 - acc: 0.3537 - val_loss: 1.2136 - val_acc: 0.4800
Epoch 2/10
141/141 [==============================] - 7059s 50s/step - loss: 1.2255 - acc: 0.4745 - val_loss: 1.1063 - val_acc: 0.5299
Epoch 3/10
141/141 [==============================] - 7061s 50s/step - loss: 1.0935 - acc: 0.5437 - val_loss: 1.0736 - val_acc: 0.5050
Epoch 4/10
 61/141 [===========>..................] - ETA: 57:13 - loss: 1.0626 - acc: 0.5423

In [ ]:
print(history_mdl1.history.keys())

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
plt.plot(history_mdl1.history['acc'])
plt.plot(history_mdl1.history['val_acc'])
plt.title('Custom Model 1 accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(history_mdl1.history['loss'])

plt.plot(history_mdl1.history['val_loss'])
plt.title('Custom Model 1 loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#Save weights
model.save_weights('cutom_model_2_v1.h5')